In [2]:
import pandas as pd
import seaborn as sns
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import plotly.offline as pyo
import plotly.graph_objs as go
import plotly.express as px
from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from scipy.stats.mstats import winsorize
from sklearn.feature_selection import VarianceThreshold 

In [3]:
def rmse(y_true, y_pred):
    """Calculate Root Mean Squared Error."""
    return np.sqrt(mean_squared_error(y_true, y_pred))

In [4]:
train = pd.read_csv("train.csv")

In [5]:
train.head()

,ID,date,cluster_id,electricity_consumption,temperature_2m_max,temperature_2m_min,apparent_temperature_max,apparent_temperature_min,sunshine_duration,daylight_duration,wind_speed_10m_max,wind_gusts_10m_max,wind_direction_10m_dominant,shortwave_radiation_sum,et0_fao_evapotranspiration
0,cluster_1_2014-01-01,2014-01-01,cluster_1,358.032,10.8,4.2,5.5,0.4,53.003333,29787.533333,40.6,79.2,186.000000,1.176667,0.483333
1,cluster_2_2014-01-01,2014-01-01,cluster_2,548.247,12.2,4.3,8.6,-0.4,8195.656667,30650.350000,36.0,81.7,170.591118,3.383333,0.736667
2,cluster_3_2014-01-01,2014-01-01,cluster_3,758.303,12.9,-0.8,10.1,-4.9,16305.260000,31547.686667,20.9,44.3,159.467752,3.880000,0.716667
3,cluster_4_2014-01-01,2014-01-01,cluster_4,1072.077,10.8,4.7,6.7,0.6,9224.803333,30769.220000,34.7,82.8,184.339753,3.153333,0.676667
4,cluster_1_2014-01-02,2014-01-02,cluster_1,386.908,10.7,7.0,6.6,3.3,22372.000000,29850.226667,34.3,75.2,210.333465,3.640000,0.780000


In [6]:
train.head()

,ID,date,cluster_id,electricity_consumption,temperature_2m_max,temperature_2m_min,apparent_temperature_max,apparent_temperature_min,sunshine_duration,daylight_duration,wind_speed_10m_max,wind_gusts_10m_max,wind_direction_10m_dominant,shortwave_radiation_sum,et0_fao_evapotranspiration
0,cluster_1_2014-01-01,2014-01-01,cluster_1,358.032,10.8,4.2,5.5,0.4,53.003333,29787.533333,40.6,79.2,186.000000,1.176667,0.483333
1,cluster_2_2014-01-01,2014-01-01,cluster_2,548.247,12.2,4.3,8.6,-0.4,8195.656667,30650.350000,36.0,81.7,170.591118,3.383333,0.736667
2,cluster_3_2014-01-01,2014-01-01,cluster_3,758.303,12.9,-0.8,10.1,-4.9,16305.260000,31547.686667,20.9,44.3,159.467752,3.880000,0.716667
3,cluster_4_2014-01-01,2014-01-01,cluster_4,1072.077,10.8,4.7,6.7,0.6,9224.803333,30769.220000,34.7,82.8,184.339753,3.153333,0.676667
4,cluster_1_2014-01-02,2014-01-02,cluster_1,386.908,10.7,7.0,6.6,3.3,22372.000000,29850.226667,34.3,75.2,210.333465,3.640000,0.780000


In [7]:
missing_percent = train.isnull().sum() / len(train) * 100
print(missing_percent)

ID                             0.0
date                           0.0
cluster_id                     0.0
electricity_consumption        0.0
temperature_2m_max             0.0
temperature_2m_min             0.0
apparent_temperature_max       0.0
apparent_temperature_min       0.0
sunshine_duration              0.0
daylight_duration              0.0
wind_speed_10m_max             0.0
wind_gusts_10m_max             0.0
wind_direction_10m_dominant    0.0
shortwave_radiation_sum        0.0
et0_fao_evapotranspiration     0.0
dtype: float64


In [8]:
train_heatmap = train.select_dtypes(exclude=['object'])

corr_matrix = train_heatmap.corr().abs()
upper = np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)
upper_matrix = corr_matrix.where(upper)
threshold = 0.9
to_drop = [column for column in upper_matrix.columns if any(upper_matrix[column] > threshold)]
train_filtered = train.drop(columns=to_drop)
print("Dropped columns due to high correlation:", to_drop)


Dropped columns due to high correlation: ['apparent_temperature_max', 'apparent_temperature_min', 'wind_gusts_10m_max', 'shortwave_radiation_sum', 'et0_fao_evapotranspiration']


In [9]:
train_filtered.head()

,ID,date,cluster_id,electricity_consumption,temperature_2m_max,temperature_2m_min,sunshine_duration,daylight_duration,wind_speed_10m_max,wind_direction_10m_dominant
0,cluster_1_2014-01-01,2014-01-01,cluster_1,358.032,10.8,4.2,53.003333,29787.533333,40.6,186.000000
1,cluster_2_2014-01-01,2014-01-01,cluster_2,548.247,12.2,4.3,8195.656667,30650.350000,36.0,170.591118
2,cluster_3_2014-01-01,2014-01-01,cluster_3,758.303,12.9,-0.8,16305.260000,31547.686667,20.9,159.467752
3,cluster_4_2014-01-01,2014-01-01,cluster_4,1072.077,10.8,4.7,9224.803333,30769.220000,34.7,184.339753
4,cluster_1_2014-01-02,2014-01-02,cluster_1,386.908,10.7,7.0,22372.000000,29850.226667,34.3,210.333465


In [10]:
train_filtered.drop(columns=['date', 'cluster_id'], inplace=True)

In [11]:
train_filtered.head(10)

,ID,electricity_consumption,temperature_2m_max,temperature_2m_min,sunshine_duration,daylight_duration,wind_speed_10m_max,wind_direction_10m_dominant
0,cluster_1_2014-01-01,358.032,10.8,4.2,53.003333,29787.533333,40.6,186.000000
1,cluster_2_2014-01-01,548.247,12.2,4.3,8195.656667,30650.350000,36.0,170.591118
2,cluster_3_2014-01-01,758.303,12.9,-0.8,16305.260000,31547.686667,20.9,159.467752
3,cluster_4_2014-01-01,1072.077,10.8,4.7,9224.803333,30769.220000,34.7,184.339753
4,cluster_1_2014-01-02,386.908,10.7,7.0,22372.000000,29850.226667,34.3,210.333465
5,cluster_2_2014-01-02,567.775,12.1,6.8,21813.283333,30708.623333,34.0,186.393300
6,cluster_3_2014-01-02,796.859,15.0,3.9,19886.503333,31601.230000,25.4,177.250439
7,cluster_4_2014-01-02,1153.619,12.5,6.6,21208.540000,30826.723333,34.0,198.722952
8,cluster_1_2014-01-03,395.319,11.7,7.1,21906.060000,29918.046667,35.3,210.000305
9,cluster_2_2014-01-03,577.312,12.1,7.0,13631.893333,30771.670000,34.9,190.006353


In [12]:
# Select numerical columns
num_cols = train_filtered.select_dtypes(include=['number'])

# Function to calculate outlier percentage using IQR
def outlier_percentage(column):
    Q1 = column.quantile(0.25)
    Q3 = column.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = ((column < lower_bound) | (column > upper_bound)).sum()
    return (outliers / len(column)) * 100  # Percentage

# Apply function to all numerical columns
outlier_percentages_train = num_cols.apply(outlier_percentage)

# Display the results
print(outlier_percentages_train.sort_values(ascending=False))

wind_speed_10m_max             1.642710
electricity_consumption        0.393566
temperature_2m_max             0.000000
temperature_2m_min             0.000000
sunshine_duration              0.000000
daylight_duration              0.000000
wind_direction_10m_dominant    0.000000
dtype: float64


In [13]:
output_filename = "train_filtered.csv"
train_filtered.to_csv(output_filename, index=False)

In [14]:
test = pd.read_csv("test.csv")

In [15]:
test.head()

,ID,date,cluster_id,temperature_2m_max,temperature_2m_min,apparent_temperature_max,apparent_temperature_min,sunshine_duration,daylight_duration,wind_speed_10m_max,wind_gusts_10m_max,wind_direction_10m_dominant,shortwave_radiation_sum,et0_fao_evapotranspiration
0,cluster_1_2022-01-01,2022-01-01,cluster_1,15.2,7.9,14.2,5.2,21902.986667,29784.506667,22.2,37.8,172.732245,4.173333,0.780000
1,cluster_2_2022-01-01,2022-01-01,cluster_2,15.9,4.1,15.3,1.3,23550.080000,30647.540000,22.7,38.5,163.972679,5.146667,0.730000
2,cluster_3_2022-01-01,2022-01-01,cluster_3,18.6,4.9,18.8,1.8,24179.643333,31545.103333,15.5,26.3,198.645137,5.713333,0.730000
3,cluster_4_2022-01-01,2022-01-01,cluster_4,13.3,3.9,11.9,1.3,23839.203333,30766.446667,16.1,27.4,183.517316,4.200000,0.486667
4,cluster_1_2022-01-02,2022-01-02,cluster_1,14.1,8.1,11.2,5.3,21018.193333,29847.110000,34.4,59.4,223.335840,4.030000,1.170000


In [16]:
test_heatmap = test.select_dtypes(exclude=['object'])

corr_matrix = test_heatmap.corr().abs()
upper = np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)
upper_matrix = corr_matrix.where(upper)
threshold = 0.9
to_drop = [column for column in upper_matrix.columns if any(upper_matrix[column] > threshold)]
test_filtered = test.drop(columns=to_drop)
print("Dropped columns due to high correlation:", to_drop)

Dropped columns due to high correlation: ['apparent_temperature_max', 'apparent_temperature_min', 'wind_gusts_10m_max', 'shortwave_radiation_sum', 'et0_fao_evapotranspiration']


In [17]:
test_filtered.drop(columns=['date', 'cluster_id'], inplace=True)

In [18]:
test_filtered.head()

,ID,temperature_2m_max,temperature_2m_min,sunshine_duration,daylight_duration,wind_speed_10m_max,wind_direction_10m_dominant
0,cluster_1_2022-01-01,15.2,7.9,21902.986667,29784.506667,22.2,172.732245
1,cluster_2_2022-01-01,15.9,4.1,23550.080000,30647.540000,22.7,163.972679
2,cluster_3_2022-01-01,18.6,4.9,24179.643333,31545.103333,15.5,198.645137
3,cluster_4_2022-01-01,13.3,3.9,23839.203333,30766.446667,16.1,183.517316
4,cluster_1_2022-01-02,14.1,8.1,21018.193333,29847.110000,34.4,223.335840


In [19]:
# Select numerical columns
num_cols = test_filtered.select_dtypes(include=['number'])

# Function to calculate outlier percentage using IQR
def outlier_percentage(column):
    Q1 = column.quantile(0.25)
    Q3 = column.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = ((column < lower_bound) | (column > upper_bound)).sum()
    return (outliers / len(column)) * 100  # Percentage

# Apply function to all numerical columns
outlier_percentages_test = num_cols.apply(outlier_percentage)

# Display the results
print(outlier_percentages_test.sort_values(ascending=False))

wind_speed_10m_max             1.294118
temperature_2m_max             0.000000
temperature_2m_min             0.000000
sunshine_duration              0.000000
daylight_duration              0.000000
wind_direction_10m_dominant    0.000000
dtype: float64


In [20]:
output_filename = "test_filtered.csv"
test_filtered.to_csv(output_filename, index=False)

In [35]:
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBRegressor

In [38]:
X_train = train_filtered.drop(columns=['electricity_consumption'])
y_train = train_filtered['electricity_consumption']

X_test = test_filtered

In [39]:
X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)

In [ ]:
# X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

In [ ]:
# dtrain = xgb.DMatrix(X_train, label=y_train)
# dtest = xgb.DMatrix(X_test)

In [33]:
param_dist = {
    'max_depth': [3, 5, 7, 10],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'n_estimators': [100, 300, 500, 800],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'gamma': [0, 1, 5],
    'min_child_weight': [1, 3, 5]
}

In [36]:
xgb = XGBRegressor(objective='reg:squarederror')

In [37]:
random_search = RandomizedSearchCV(
    xgb, param_distributions=param_dist, 
    n_iter=50, scoring='neg_root_mean_squared_error',
    cv=3, verbose=1, random_state=42, n_jobs=-1
)

In [40]:
random_search.fit(X_train, y_train)
print("Best Params:", random_search.best_params_)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


KeyboardInterrupt: 

In [ ]:
# evallist = [(dtrain, 'train')]
# num_round = 100
# bst = xgb.train(params, dtrain, num_round, evallist)

c:\Users\HP\anaconda3\Lib\site-packages\xgboost\core.py:723: FutureWarning:

Pass `evals` as keyword args.



[0]	train-rmse:281.54892
[1]	train-rmse:277.33206
[2]	train-rmse:273.84918
[3]	train-rmse:269.95211
[4]	train-rmse:266.65235
[5]	train-rmse:264.42663
[6]	train-rmse:261.84091
[7]	train-rmse:259.64818
[8]	train-rmse:257.63093
[9]	train-rmse:256.16807
[10]	train-rmse:254.92757
[11]	train-rmse:253.78914
[12]	train-rmse:252.60342
[13]	train-rmse:251.71421
[14]	train-rmse:250.91380
[15]	train-rmse:250.38749
[16]	train-rmse:249.60531
[17]	train-rmse:248.91204
[18]	train-rmse:248.30040
[19]	train-rmse:247.77381
[20]	train-rmse:247.20121
[21]	train-rmse:246.63194
[22]	train-rmse:246.17848
[23]	train-rmse:245.79726
[24]	train-rmse:245.54050
[25]	train-rmse:245.14277
[26]	train-rmse:244.76720
[27]	train-rmse:244.32745
[28]	train-rmse:243.95906
[29]	train-rmse:243.72900
[30]	train-rmse:243.26695
[31]	train-rmse:243.00460
[32]	train-rmse:242.78665
[33]	train-rmse:242.42442
[34]	train-rmse:242.08882
[35]	train-rmse:241.84626
[36]	train-rmse:241.53294
[37]	train-rmse:241.26219
[38]	train-rmse:241.03

In [29]:
y_pred = bst.predict(dtest)

In [30]:
submission = pd.DataFrame({
    'ID': test_filtered['ID'],  # Ganti sesuai kolom unik jika ada
    'electricity_consumption': y_pred
})

In [ ]:
submission.to_csv('submission.csv', index=False)

File submission.csv berhasil disimpan.
